In [6]:
import ipaddress
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

# Read the dataset
df = pd.read_csv("C:/Users/Lenovo/Downloads/Darknet.csv")

# Drop unnecessary columns
df = df.drop(["Flow ID", "Timestamp", "Label2"], axis=1)

# Drop rows with missing values
df = df.dropna()

# Convert IP addresses to integers
df['Src IP'] = df['Src IP'].apply(lambda x: int(ipaddress.ip_address(x)))
df['Dst IP'] = df['Dst IP'].apply(lambda x: int(ipaddress.ip_address(x)))

# Encode the label column
label_encoder = LabelEncoder()
df['Label1'] = label_encoder.fit_transform(df['Label1'])

# Save the preprocessed dataset
df.to_csv("processed.csv", index=False)

# Identify and handle any problematic values in the dataset
problematic_columns = []
for col in df.columns:
    if df[col].dtype == np.float64 or df[col].dtype == np.int64:
        max_value = df[col].max()
        min_value = df[col].min()
        if max_value == np.inf or min_value == -np.inf:
            problematic_columns.append(col)

# Remove rows with problematic values
if problematic_columns:
    print("Removing rows with problematic values...")
    df = df[~df[problematic_columns].isin([np.inf, -np.inf]).any(axis=1)]

# Split features and label
features = df.drop(['Label1'], axis=1)
label = df['Label1']

# Scale the features using Min-Max scaling
scaler = MinMaxScaler()
scaled_features = scaler.fit_transform(features)

# Save scaled dataset
scaled_df = pd.DataFrame(scaled_features, columns=features.columns)
scaled_df['Label1'] = label
scaled_df.to_csv("scaled3.csv", index=False)

# Split the dataset into train and test sets
X_train, X_test, y_train, y_test = train_test_split(scaled_features, label, test_size=0.2, random_state=42)

# Build the model
model = tf.keras.Sequential([
    tf.keras.layers.Dense(250, activation='relu', input_shape=(scaled_features.shape[1],)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(4, activation='softmax')
])

# Compile 
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Train
model.fit(X_train, y_train, epochs=10, batch_size=256, validation_data=(X_test, y_test))

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)

# Printing the accuracy
print(f'Test Loss: {loss:.4f}')
print(f'Test Accuracy: {accuracy:.4f}')

with open('accuracy3.txt', 'w') as f:
    f.write(f'Test Loss: {loss:.4f}\n')
    f.write(f'Test Accuracy: {accuracy:.4f}\n')

Removing rows with problematic values...
Epoch 1/10


C:\Users\Lenovo\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:88: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


496/496 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accuracy: 0.8946 - loss: 0.3443 - val_accuracy: 0.9866 - val_loss: 0.0516
Epoch 2/10
496/496 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.9888 - loss: 0.0428 - val_accuracy: 0.9885 - val_loss: 0.0405
Epoch 3/10
496/496 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.9902 - loss: 0.0358 - val_accuracy: 0.9908 - val_loss: 0.0378
Epoch 4/10
496/496 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.9914 - loss: 0.0328 - val_accuracy: 0.9905 - val_loss: 0.0346
Epoch 5/10
496/496 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.9913 - loss: 0.0323 - val_accuracy: 0.9913 - val_loss: 0.0322
Epoch 6/10
496/496 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accuracy: 0.9921 - loss: 0.0285 - val_accuracy: 0.9905 - val_loss: 0.0321
Epoch 7/10
496/496 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.9927 - loss: 0.0272 - val_accuracy: 0.9911 - val_loss: 0.0303
Epoch 8/10
496/496 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.9929 - loss: 0.0268 - val_accuracy: 0.9917 - val_

In [7]:
# Load the scaled dataset
scaled_df = pd.read_csv("scaled3.csv")

# Display the first few rows of the dataset
print(scaled_df.head())

     Src IP  Src Port    Dst IP  Dst Port  Protocol  Flow Duration  \
0  0.047326  0.872188  0.844420  0.006760  0.352941       0.000002   
1  0.047326  0.872204  0.844420  0.006760  0.352941       0.000003   
2  0.047326  0.872219  0.844420  0.006760  0.352941       0.000004   
3  0.047326  0.749748  0.289938  0.006760  0.352941       0.000003   
4  0.047326  0.529450  0.678274  0.294575  0.352941       0.089820   

   Total Fwd Packet  Total Bwd packets  Total Length of Fwd Packet  \
0          0.000000           0.000002                    0.000000   
1          0.000000           0.000002                    0.000000   
2          0.000000           0.000002                    0.000000   
3          0.000000           0.000002                    0.000000   
4          0.002477           0.000850                    0.000084   

   Total Length of Bwd Packet  ...  Fwd Seg Size Min  Active Mean  Active Std  \
0                     0.00000  ...          0.454545          0.0         0.0

In [10]:
import os

# Get the current working directory
current_directory = os.getcwd()

# Check if the file exists in the current directory or its subdirectories
file_found = False
for root, dirs, files in os.walk(current_directory):
    if "scaled3.csv" in files:
        file_path = os.path.join(root, "scaled3.csv")
        print(f"The 'scaled3.csv' file is located at: {file_path}")
        file_found = True
        break

if not file_found:
    print("The 'scaled3.csv' file was not found in the current directory or its subdirectories.")

The 'scaled3.csv' file is located at: C:\Users\Lenovo\scaled3.csv
